In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [3]:
# This is the path.py library, included in the Anaconda distribution of Python
from path import Path as p

In [4]:
PATH = "../../../data/dogscats/"
sz=224
bs = 64
n = 1000 # Number of dogs and cats to train model on
n_cats = n
n_dogs = n # We'll set these variables differently in the future for testing
# imbalanced classes
CAT = 0 # Dummy class variables
DOG = 1 # Dummy class variables

In [4]:
torch.cuda.is_available()

True

In [5]:
p(PATH).dirs()

[Path('../../../data/dogscats/models'),
 Path('../../../data/dogscats/sample'),
 Path('../../../data/dogscats/test1'),
 Path('../../../data/dogscats/tmp'),
 Path('../../../data/dogscats/train'),
 Path('../../../data/dogscats/valid')]

In [11]:
print(p(PATH + "train/").dirs())
print(p(PATH + "valid/").dirs())

[Path('../../../data/dogscats/train/cats'), Path('../../../data/dogscats/train/dogs')]
[Path('../../../data/dogscats/valid/cats'), Path('../../../data/dogscats/valid/dogs')]


In [5]:
# Function to create relative paths for the cat and dog files
def get_names(filelist, suffix):
    return [suffix + f.name for f in filelist]

In [6]:
# Now we create lists of the filenames of the cats and dogs
train_cats = get_names(p(PATH + "train/cats/").files(), "train/cats/")
train_dogs = get_names(p(PATH + "train/dogs/").files(), "train/dogs/")
valid_cats = get_names(p(PATH + "valid/cats/").files(), "valid/cats/")
valid_dogs = get_names(p(PATH + "valid/dogs/").files(), "valid/dogs/")
print(len(train_cats))
print(len(train_dogs))
print(len(valid_cats))
print(len(valid_dogs))

11500
11500
1000
1000


In [7]:
# We generate the input variables for our model later on
p(PATH + "tmp/").rmtree_p()
catfiles = list(np.random.choice(train_cats, n_cats, False)) + list(valid_cats)
dogfiles = list(np.random.choice(train_dogs, n_dogs, False)) + list(valid_dogs)
fnames = catfiles + dogfiles
y = np.array([CAT]*len(catfiles) + [DOG]*len(dogfiles))
classes = list(set(y))
valid_cat_idxs = range(n_cats, n_cats + len(valid_cats))
print(len(valid_cat_idxs))
valid_dog_idxs = range(len(catfiles) + n_dogs, len(fnames))
print(len(valid_dog_idxs))
val_idxs = list(valid_cat_idxs) + list(valid_dog_idxs)
                                               
# fnames: file names
# y: numpy array which contains target labels ordered by filenames.
# classes: a list of all labels/classifications, [0, 1]
# val_idxs: index of images to be used for validation.

1000
1000


In [8]:
# Let's set up our model using the pretrained Resnet34 Imagenet model
arch=resnet34
data = ImageClassifierData.from_names_and_array(PATH, fnames, y, classes, \
                                                val_idxs, bs=bs, \
                                                tfms=tfms_from_model(arch, sz))
learn = ConvLearner.pretrained(arch, data, precompute=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [00:21<00:00,  1.52it/s]


In [9]:
# Use a learning rate of 0.01 and train for 5 epochs
lr = 0.01
epochs = 5
learn.fit(lr, epochs)

epoch      trn_loss   val_loss   accuracy                                                                              
    0      0.510213   0.108337   0.969     
    1      0.290492   0.159066   0.938                                                                                 
    2      0.190142   0.078571   0.973                                                                                 
    3      0.13592    0.098637   0.963                                                                                 
    4      0.100219   0.100527   0.9615                                                                                



[array([0.10053]), 0.9615]